#必做项

搭载Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
path="/content/drive/MyDrive/NeuralRecon-W-test"
os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['metrics.py',
 'README.md',
 'LICENSE',
 'losses.py',
 'environment.yaml',
 'tools',
 'scripts',
 'utils',
 'models',
 'evaluations',
 'assets',
 'rendering',
 'opt.py',
 'lightning_modules',
 'datasets',
 '.gitignore',
 'data',
 'weights',
 'log',
 'config',
 'logs',
 '__pycache__',
 'results',
 'requirements.txt',
 'kaolin',
 'train.py',
 '\\']

# 基础操作

##查看GPU信息

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Aug  3 05:27:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 查看pip列表

In [ ]:
!pip list

Package                       Version                      Location
----------------------------- ---------------------------- ------------------------------------------------
absl-py                       1.2.0
addict                        2.4.0
aiobotocore                   2.1.2
aiohttp                       3.8.1
aioitertools                  0.10.0
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
anyio                         3.6.1
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
asgiref                       3.5.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                    

##查看某个安装包版本

In [ ]:
!pip show pytorch_lightning

## 安装特定版本

In [ ]:
!pip install pytorch_lightning==1.4.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch_lightning-1.4.8-py3-none-any.whl (924 kB)
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.6.5
    Uninstalling pytorch-lightning-1.6.5:
      Successfully uninstalled pytorch-lightning-1.6.5


## 更新某包

In [ ]:
!pip install --upgrade pytorch_lightning

## cuda/python版本

验证版本：Python >= 3.7, <= 3.9；CUDA >= 10.0 (with ‘nvcc’ installed)均满足

In [ ]:
!python --version
!nvcc -V
import torch
torch.version.cuda

Python 3.7.13
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


'10.2'

##Restart Runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

# 依赖包安装

## 一键安装

In [ ]:
!pip install open3d==0.12.0
!pip install kornia==0.4.1
!pip install loguru
!pip install torch_optimizer
!pip install trimesh==3.9.1
!pip install cython==0.29.20
!pip install lpips==0.1.3
!pip install torchmetrics==0.7.0
!pip install yacs
!pip install test-tube==0.7.5
!pip install tornado==5.1.0
#%pip install git+https://github.com/cpfy/lightning
%pip install git+https://github.com/cpfy/lightning@v1.4.8_origin

import os
path="/content/drive/MyDrive/NeuralRecon-W-test/kaolin"
os.chdir(path)
%env IGNORE_TORCH_VER=1
!python setup.py develop
path="/content/drive/MyDrive/NeuralRecon-W-test"
os.chdir(path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188.4 MB 26 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 225 kB 8.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 440 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 628 kB 8.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.1 MB/s 
  Attempting uninstall: cython
    Found existing installation: C

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cpfy/lightning (to revision v1.4.8_origin) to /tmp/pip-req-build-zjoo7ee7
  Running command git clone -q https://github.com/cpfy/lightning /tmp/pip-req-build-zjoo7ee7
  Running command git checkout -b v1.4.8_origin --track origin/v1.4.8_origin
  Switched to a new branch 'v1.4.8_origin'
  Branch 'v1.4.8_origin' set up to track remote branch 'v1.4.8_origin' from 'origin'.
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 829 kB 8.8 MB/s 
     |████████████████████████████████| 596 kB 64.3 MB/s 
     |████████████████████████████████| 141 kB 100.3 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-1.4.8-py3-none-any.whl size=924082 sha256=e170066833e3b472c1313b9eb0dbf6

##torch系列安装（废弃）

补充requirements包，下一步之前先如下安装

In [ ]:
!pip install torch==1.12.0+cu113 torchvision==0.13.0 torchtext==0.13.0 torchaudio==0.12.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |██████████████▌                 | 834.1 MB 1.2 MB/s eta 0:13:50tcmalloc: large alloc 1147494400 bytes == 0x390e6000 @  0x7f0035ab0615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |██████████████████▍             | 1055.7 MB 1.2 MB/s eta 0:10:33tcmalloc: large alloc 1434370048 bytes == 0x7d73c000 @  0x7f0035ab0615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

安装1.11.0版本（后废弃）

In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0 torchtext==0.12.0 torchaudio==0.11.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████▎               | 834.1 MB 97.8 MB/s eta 0:00:09tcmalloc: large alloc 1147494400 bytes == 0x39618000 @  0x7f4993265615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████▋           | 1055.7 MB 1.2 MB/s eta 0:08:19tcmalloc: large alloc 1434370048 bytes == 0x7dc6e000 @  0x7f4993265615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████

##kaolin安装

* 注意不可用pip安装 `pip install kaolin` ，会有依赖报错

* 不可`python kaolin/setup.py develop`，这里必须有一步cd kaolin，调用子目录会报错某文件找不到

* 环境变量`export`配置的替代方案：https://stackoverflow.com/questions/53306150/setting-environment-variables-in-google-colab

In [ ]:
#已下载
#!git clone --recursive https://github.com/NVIDIAGameWorks/kaolin

!pip install cython==0.29.20

path="/content/drive/MyDrive/NeuralRecon-W-test/kaolin"
os.chdir(path)
%env IGNORE_TORCH_VER=1
!python setup.py develop

path="/content/drive/MyDrive/NeuralRecon-W-test"
os.chdir(path)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.2 MB/s 
  Attempting uninstall: cython
    Found existing installation: Cython 0.29.31
    Uninstalling Cython-0.29.31:
      Successfully uninstalled Cython-0.29.31
env: IGNORE_TORCH_VER=1
setup.py:37: UserWarning: Kaolin is compatible with PyTorch >=1.5.0, <=1.11.0, but found version 1.12.0+cu113. Continuing with the installed version as IGNORE_TORCH_VER is set.
  f'Kaolin is compatible with PyTorch >={TORCH_MIN_VER}, <={TORCH_MAX_VER}, '
Compiling kaolin/cython/ops/mesh/triangle_hash.pyx because it depends on /usr/local/lib/python3.7/dist-packages/Cython/Includes/libcpp/vector.pxd.
Compiling kaolin/cython/ops/conversions/mise.pyx because it depends on /usr/local/lib/python3.7/dist-packages/Cython/Includes/libcpp/vector.pxd.
[1/2] Cythonizing kaolin/cython/ops/conversions/mise.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Mai

本来应用于生成ray cache数据，目前kaolin一直报错

提了一个Issue：https://github.com/NVIDIAGameWorks/kaolin/issues/597

## 十步pip安装

In [ ]:
!pip install open3d==0.12.0
!pip install kornia==0.4.1
!pip install loguru
!pip install torch_optimizer
!pip install trimesh==3.9.1
#!pip install kaolin
!pip install cython==0.29.20
!pip install tornado==5.1.0

!pip install lpips==0.1.3
!pip install torchmetrics==0.7.0
!pip install yacs
!pip install test-tube==0.7.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188.4 MB 25 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 225 kB 4.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 2.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 412 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 628 kB 4.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.2 MB/s 
  Attempting uninstall: cython
    Found existing installation: C

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 396 kB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for test-tube: filename=test_tube-0.7.5-py3-none-any.whl size=25356 sha256=6143c514debbb6162cf7d81f1aa4582008bef22415ac4bcb2cbfc7dbe6320e7b
  Stored in directory: /root/.cache/pip/wheels/1c/50/0d/15b3236957cc18a5c39ec4d4d4d21624f4d4a876756ec17064
Successfully built test-tube


In [ ]:
!sh scripts/data_generation.sh data/heritage-recon/lincoln_memorial
!sh scripts/data_generation.sh data/heritage-recon/palacio_de_bellas_artes
!sh scripts/data_generation.sh data/heritage-recon/pantheon_exterior

^C
/bin/bash: shscripts/data_generation.sh: No such file or directory
+ set -u
+ set -e
+ date +%Y%m%d_%H%M%S
+ now=20220729_082847
+ pwd
+ echo working directory is /content/drive/MyDrive/NeuralRecon-W-test
working directory is /content/drive/MyDrive/NeuralRecon-W-test
+ jobname=data-generation-data/heritage-recon/pantheon_exterior-20220729_082847
+ export CUDA_VISIBLE_DEVICES=0
+ dataset_name=phototourism
+ cache_dir=cache_sgs
+ root_dir=data/heritage-recon/pantheon_exterior
+ min_observation=-1
+ [ ! -f data/heritage-recon/pantheon_exterior/pantheon_exterior.tsv ]
+ python tools/prepare_data/prepare_data_cache.py --root_dir data/heritage-recon/pantheon_exterior --dataset_name phototourism --cache_dir cache_sgs --img_downscale 1 --semantic_map_path semantic_maps --split_to_chunks 64
+ tee log/data-generation-data/heritage-recon/pantheon_exterior-20220729_082847.log
INFO - 2022-07-29 08:28:49,282 - utils - NumExpr defaulting to 4 threads.
INFO - 2022-07-29 08:28:50,804 - rasterization

## lightning安装

自己fork的稳定版

In [ ]:
#%pip install git+https://github.com/cpfy/lightning
%pip install git+https://github.com/cpfy/lightning@v1.4.8_origin
#（08.03更新）自己改的有隐患，回退到1.4.8版本小幅修改

pytorch-lightning 要求从Github下载

见：https://stackoverflow.com/questions/66538407/unable-to-import-pytorch-lightning-on-google-colab

另：https://github.com/Lightning-AI/lightning/issues/6210

In [ ]:
%pip install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path

In [ ]:
%pip install git+https://github.com/cpfy/lightning
import pytorch_lightning as pl
print(pl.__version__)
from pytorch_lightning.loggers import TestTubeLogger

In [ ]:
%pip install git+https://github.com/Lightning-AI/lightning
import pytorch_lightning as pl
print(pl.__version__)

In [ ]:
import pytorch_lightning as pl
class MyDataModule(pl.LightningDataModule):
  pass
#print(pl.__version__)

# 第一次初始化操作

操作说明与仓库地址：https://github.com/zju3dv/NeuralRecon-W

## 下载数据

下载数据，示例见：（已完成）

```shell
mkdir data && cd data
gdown --id 1eZvmk4GQkrRKUNZpagZEIY_z8Lsdw94v
```

此处参见unzipfile.ipynb

## <核心>场景cache生成

对所有scene生成Ray Cache，例如：

```python
for SCENE_NAME in brandenburg_gate lincoln_memorial palacio_de_bellas_artes pantheon_exterior; do
  scripts/data_generation.sh data/heritage-recon/${SCENE_NAME}
done
```

看起来似乎无论如何都会爆RAM，尝试TPU的35G RAM（不行，报错）


In [ ]:
!sh scripts/data_generation.sh data/heritage-recon/brandenburg_gate

In [ ]:
!sh scripts/data_generation.sh data/heritage-recon/lincoln_memorial

In [ ]:
!sh scripts/data_generation.sh data/heritage-recon/palacio_de_bellas_artes

In [ ]:
!sh scripts/data_generation.sh data/heritage-recon/pantheon_exterior

## 高RAM榨取

In [ ]:
[1]*10**10

# 模型训练

##安装依赖库

目前有极大隐患

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 924 kB 4.3 MB/s 
     |████████████████████████████████| 50.4 MB 100 kB/s 
     |████████████████████████████████| 11.6 MB 23.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 26.9 MB 1.3 MB/s 
     |████████████████████████████████| 9.5 MB 47.2 MB/s 
     |████████████████████████████████| 13.2 MB 53.1 MB/s 
     |████████████████████████████████| 265 kB 52.0 MB/s 
     |████████████████████████████████| 105 kB 63.9 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
     |████████████████████████████████| 1.6 MB 35.5 MB/s 
     |████████████████████████████████| 563 kB 50.5 MB/s 
     |████████████████████████████████| 817 kB 29.4 MB/s 
     |████████████████████████████████| 829 kB 58.3 MB/s 
     |████████████████████████████████| 190 kB 48.9 MB/s 
     |████████████████████████

##<核心>运行训练脚本

看起来似乎无论如何都会爆RAM

尝试 yaml中配置 use_cache=False（并没有啥用，只是重复一遍data_generation）

yaml中配置IMG_DOWNSCALE=4可解决RAM不足问题

In [8]:
!sh scripts/train.sh exp_BG config/train_brandenburg_gate.yaml 1 1

流式输出内容被截断，只能显示最后 5000 行内容。

100%|██████████| 1/1 [00:00<00:00, 1045.70it/s]
[WARNING] batch has 0 intersections!!

100%|██████████| 1/1 [00:00<00:00, 1113.43it/s]
[WARNING] batch has 0 intersections!!

100%|██████████| 1/1 [00:00<00:00, 375.03it/s]

100%|██████████| 1/1 [00:00<00:00, 343.85it/s]

100%|██████████| 1/1 [00:00<00:00, 1151.02it/s]
[WARNING] batch has 0 intersections!!

100%|██████████| 1/1 [00:00<00:00, 376.10it/s]

100%|██████████| 1/1 [00:00<00:00, 411.13it/s]

100%|██████████| 1/1 [00:00<00:00, 386.57it/s]
padding valid depth percentage: from 0.0004333131120547708 to 0.2 with padding 2879
sample depth percent after padding: 0.1826171875

100%|██████████| 1/1 [00:00<00:00, 390.97it/s]

100%|██████████| 1/1 [00:00<00:00, 353.83it/s]

100%|██████████| 1/1 [00:00<00:00, 374.73it/s]
padding valid depth percentage: from 0.0007470405700494196 to 0.2 with padding 4335
sample depth percent after padding: 0.197265625

100%|██████████| 1/1 [00:00<00:00, 332.88it/s]

 34%|███▍     

训练方法2：不使用config.yaml，独立设置参数

In [ ]:
#!python train.py --cfg_path config/train_brandenburg_gate.yaml --num_gpus 1 --num_nodes 2 --num_epochs 20 --batch_size 2048 --test_batch_size 512 --num_workers 16 --exp_name train-exp_BG
!python train.py --cfg_path

INFO - 2022-07-31 08:20:28,593 - utils - NumExpr defaulting to 4 threads.
INFO - 2022-07-31 08:20:30,069 - rasterization - Cannot import nvdiffrast
INFO - 2022-07-31 08:20:30,441 - instantiator - Created a temporary directory at /tmp/tmp5ihlrb95
INFO - 2022-07-31 08:20:30,441 - instantiator - Writing /tmp/tmp5ihlrb95/_remote_module_non_scriptable.py
Traceback (most recent call last):
  File "train.py", line 12, in <module>
    from pytorch_lightning.loggers import TestTubeLogger
ImportError: cannot import name 'TestTubeLogger' from 'pytorch_lightning.loggers' (/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/__init__.py)


#Evaluating

评估已有weights

In [ ]:
#!sh scripts/sdf_extract.sh $EXP_NAME config/train_${SCENE_NAME}.yaml $CKPT_PATH 10
!sh scripts/sdf_extract.sh exp_BG config/train_brandenburg_gate.yaml /weights/deeplabv3_r101-d8_512x512_160k_ade20k_20200615_105816-b1f72b3b 1

^C


##evaluation pipeline

In [ ]:
!sh scripts/eval_pipeline.sh $SCENE_NAME $MESH_PATH

#Reconstructing

In [ ]:
!sh scripts/preprocess_data.sh

+ set -u
+ set -e
+ date +%Y%m%d_%H%M%S
+ now=20220728_141553
+ pwd
+ echo working directory is /content/drive/MyDrive/NeuralRecon-W-test
working directory is /content/drive/MyDrive/NeuralRecon-W-test
+ jobname=data-generation-20220728_141553
+ SRC_PATH=/data/phototourism/training_set
+ SCENE_NAME=brandenburg_gate
+ DEST_PATH=/data/recon_data
+ COLMAP_PATH=dense/sparse
+ IMG_PATH=dense/images
+ SPLIT_TYPE=none
+ python tools/pre_process.py --src /data/phototourism/training_set/brandenburg_gate --dest /data/recon_data --split none --img_dir dense/images --colmap_dir dense/sparse
/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_hip.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
images groups:  [['']]
Traceback (most recent call last):
  File "tools/pre_process.py", line 193, in <module>
    copy_file(args.src, args.dest, group_list, args.colmap

# 测试

In [ ]:
import torchtext

## 易错包检验

In [5]:
import torchtext

In [ ]:
from torchtext.legacy.data import Batch

===分割线===

In [ ]:
!python train.py --cfg_path config/train_brandenburg_gate.yaml --num_gpus 1 --num_nodes 2 --num_epochs 20 --batch_size 2048 --test_batch_size 512 --num_workers 16 --exp_name train-exp_BG

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_hip.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
INFO - 2022-07-28 08:54:41,425 - rasterization - Cannot import nvdiffrast
src/tcmalloc.cc:283] Attempt to free invalid pointer 0x7ffd491a5198 


In [ ]:
!pip install tensorboard==2.8
!pip install requests==2.23.0
!pip install folium==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.3 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=18f3edeeef5d5875d9093334ab5792c04a79cdd117391f909c02151171bd5530
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
!pip show pytorch_lightning

In [6]:
import pytorch_lightning

In [7]:
import pytorch_lightning.loggers

In [ ]:
import torch
torch.__config__.show()

'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.3\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86\n  - CuDNN 8.3.2  (built against CUDA 11.5)\n  - Magma 2.5.2\n  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.3, CUDNN_VERSION=8.3.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDE

In [ ]:
from pytorch_lightning.loggers import TestTubeLogger

In [ ]:
import kaolin

In [ ]:
from kaolin import _C

In [ ]:
import pytorch_lightning as pl

## TPU&cuda尝试

### 安装TPU+torch兼容环境

某个看起来很官方的教程：https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/mnist-training.ipynb#scrollTo=OApBOAe1fpH_

需要用到一个叫xla的辅助？

In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 187.4 MB 30 kB/s 
     |████████████████████████████████| 57 kB 2.9 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.317 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [ ]:
import torch
import torchvision
print(torch.cuda.is_available())

a = torch.Tensor(5,3)
a=a.cuda()
print(a)


TPU测试

In [ ]:
import os
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

TPU address is grpc://10.45.155.250:8470
